In [ ]:
%%capture
import delfi.distribution as dd
import delfi.utils.io as io
import numpy as np
import pickle

import delfi.inference as infer
import delfi.generator as dg
from delfi.simulator import GaussMixture
import delfi.summarystats as ds
from lfimodels.abc_methods.run_abc import run_smc

def run_panel_a(algo, seed, eps_init=[2,0.5,0.025], n_particles=5000, maxsim=1e6):
    n_params = 1
    m = GaussMixture(dim=n_params, bimodal=False, noise_cov=[1.0, 0.01], seed=seed)
    p = dd.Uniform(lower=[-10], upper=[10], seed=seed)
    s = ds.Identity()
    g = dg.Default(model=m, prior=p, summary=s)

    obs = np.array([[0.]])
    kwargs = {'model' : m,
              'prior' : p,
              'summary' : s, 
        
              'obs_stats': obs.copy(),
              'n_particles' : n_particles, 
              'eps_init' : eps_init, 
              'maxsim' : maxsim,
              'fn' : None,
              'seed': seed}
                     
    if algo == 'SMC': 
        out = run_smc(**kwargs)
    elif algo == 'SMC_regAdj':
        out = run_smc(**kwargs, regAdj = True)
    elif algo == 'SMC_AW':
        out = run_smc(**kwargs, mode='AW')
    elif algo == 'SMC_OW':
        out = run_smc(**kwargs, mode='OW')
    
    all_ps, all_xs, all_logweights, all_eps, all_nsims = out
    posterior = all_ps[-1]
    
    return {'algo': algo,
            'seed': seed,
            'posterior': posterior,
            'all_ps' : all_ps,
            'all_xs' : all_xs,
            'all_logweights' : all_logweights,
            'all_eps' : all_eps,
            'all_nsims' : all_nsims,
            'obs' : obs,
            'kwargs' : kwargs
           }


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

p_true = dd.MoG(a=[0.5, 0.5], ms=[np.asarray([0.]), np.asarray([0.])], Ss=[1.0*np.eye(1), 0.01*np.eye(1)])

seed = 42

m = GaussMixture(dim=1, bimodal=False, noise_cov=[1.0, 0.01], seed=seed)
p = dd.Uniform(lower=[-10], upper=[10], seed=seed)
s = ds.Identity()
g = dg.Default(model=m, prior=p, summary=s)
trn_data = g.gen(1000)
plt.subplot(1,2,1)
plt.plot(trn_data[1], trn_data[0], '.')

m = GaussMixture(dim=1, bimodal=False, noise_cov=[1.0, 0.01], seed=seed)
p = p_true
s = ds.Identity()
g = dg.Default(model=m, prior=p, summary=s)
trn_data = g.gen(1000)
plt.subplot(1,2,2)
plt.plot(trn_data[1], trn_data[0], '.')

plt.show()

In [ ]:
"""
seeds = range(90, 91)
all_raw, all_AW, all_OW = [], [], []
for seed in seeds:
    
    print('\n')
    print('\n seed #' + str(seed))        
    print('\n')        
    
    
    for [algo, all_res] in zip(['SMC', 'SMC_AW', 'SMC_OW'], [all_raw, all_AW, all_OW]):

        print('\n algo ' + str(algo))
        
        res = run_panel_a(algo=algo, eps_init=[2,0.5,0.025], seed=seed, n_particles=500)
        
        print('eps levels \n', 
              res['kwargs']['eps_init'])
        print('avg. #sim steps / accepted particle \n', 
              np.array(res['all_nsims'])/res['kwargs']['n_particles'])    
        ess = np.zeros(3)
        for r in range(3):
            w = np.exp(res['all_logweights'][r])
            w = w / w.sum()
            ess[r] = 1./(np.sum(w**2) * w.size)
        print('avg. #sim steps / eff.acc. particle \n', 
              np.array(res['all_nsims'])/(ess*res['kwargs']['n_particles']))    
        
        all_res.append(res)   
    
        print('\n \n')

"""

In [ ]:
seeds = range(90, 95)
all_raw, all_ra, all_OW = [], [], []
for seed in seeds:
    
    print('\n')
    print('\n seed #' + str(seed))        
    print('\n')        
    
    
    for [algo, all_res] in zip(['SMC', 'SMC_regAdj'], [all_raw, all_ra]):

        print('\n algo ' + str(algo))
        
        res = run_panel_a(algo=algo, eps_init=[2,0.5,0.025], seed=seed, n_particles=1000)
        
        print('eps levels \n', 
              res['kwargs']['eps_init'])
        print('avg. #sim steps / accepted particle \n', 
              np.array(res['all_nsims'])/res['kwargs']['n_particles'])    
        ess = np.zeros(3)
        for r in range(3):
            w = np.exp(res['all_logweights'][r])
            w = w / w.sum()
            ess[r] = 1./(np.sum(w**2) * w.size)
        print('avg. #sim steps / eff.acc. particle \n', 
              np.array(res['all_nsims'])/(ess*res['kwargs']['n_particles']))    
        
        all_res.append(res)   
    
        print('\n \n')



In [ ]:
ptrue = dd.MoG(a=0.5*np.ones(2),
               ms=[np.zeros(1), np.zeros(1)],
               Ss=[np.eye(1), 0.01*np.eye(1)])

xx = np.linspace(-1.5,1.5,300).reshape(-1,1)
plt.plot(xx, ptrue.eval(xx, log=False))
plt.show()

In [ ]:
algos = ['SMC', 'SMC_regAdj']

ess =   np.zeros((len(algos),3,len(seeds)))
nsims = np.zeros((len(algos),3,len(seeds)))
n_particles = np.zeros((len(algos),3,len(seeds)))
ll    = np.zeros((len(algos),3,len(seeds)))

for i in range(len(seeds)):
    for a in range(len(algos)):
        
        res = [all_raw, all_ra][a][i]

        nsims[a, :, i] = res['all_nsims']
        n_particles[a, :, i] = res['kwargs']['n_particles']
        for r in range(3):
            w = np.exp(res['all_logweights'][r])
            w = w / w.sum()
            ess[a,r,i] = 1./(np.sum(w**2) * w.size)
            ll[a,r,i] = np.sum(ptrue.eval(res['all_ps'][r],log=True))


In [ ]:
plt.figure(figsize=(16,5))
for r in range(3):
    plt.subplot(1,3,r+1)
    plt.imshow(ess[:,r,:].T, interpolation='None', aspect='auto')
    plt.colorbar()
    print('\n r = ' + str(r+1) + '\n')
    print(np.mean(ess[:,r,:],axis=1))
    plt.title('round r = ' + str(r+1))
plt.suptitle('ESS')
plt.show()

In [ ]:
plt.figure(figsize=(16,5))
for r in range(3):
    plt.subplot(1,3,r+1)
    plt.imshow(nsims[:,r,].T, interpolation='None', aspect='auto')
    plt.colorbar()
    print('\n r = ' + str(r+1) + '\n')
    print(np.mean(nsims[:,r,:],axis=1))
    plt.title('round r = ' + str(r+1))
plt.suptitle('nsims')
plt.show()

In [ ]:
plt.figure(figsize=(16,5))
for r in range(3):
    plt.subplot(1,3,r+1)
    tmp = nsims[:,r,:].T/(ess[:,r,:]*n_particles[:,r,:]).T
    plt.imshow(tmp, interpolation='None', aspect='auto')
    plt.colorbar()
    print('\n r = ' + str(r+1) + '\n')
    print(np.mean(tmp.T,axis=1))
    plt.title('round r = ' + str(r+1))
plt.suptitle('nsims / ess')
plt.show()

In [ ]:
plt.figure(figsize=(16,5))
for r in range(3):
    plt.subplot(1,3,r+1)
    tmp = ll[:,r,:].T
    plt.imshow(tmp, interpolation='None', aspect='auto')
    plt.colorbar()
    print('\n r = ' + str(r+1) + '\n')
    print(np.mean(tmp.T,axis=1))
    plt.title('round r = ' + str(r+1))
plt.suptitle('log-probability of posterior draw under gt distr.')
plt.show()

In [ ]:
import scipy
from lfimodels.abc_methods.run_abc import set_kernel_bandwidths
plt.figure(figsize=(16,5))

xx = np.linspace(-3,3,300)

for i in range(len(seeds)):
    for a in range(len(algos)):
        
        plt.subplot(1,3,a+1)
        res = [all_raw, all_ra][a][i]
        bw = set_kernel_bandwidths(data=res['posterior'], 
                                   weights=np.exp(res['all_logweights'][-1]), 
                                   obs_stats=res['obs'], 
                                   rule='of_thumb')[0,0]
        #kde = scipy.stats.gaussian_kde(res['posterior'].T, bw)
        #plt.plot(xx,kde.pdf(xx.reshape(1,-1)), 'k')
        
        kde = dd.MoG(a=np.exp(res['all_logweights'][-1]), 
                     ms=[res['posterior'][ii] for ii in range(res['posterior'].size)],
                     Ss=[  np.atleast_2d(bw**2)  for ii in range(res['posterior'].size)]
                    )
        plt.plot(xx,kde.eval(xx.reshape(-1,1), log=False), 'k')

for a in range(len(algos)):

    plt.subplot(1,3,a+1)
    plt.plot(xx, ptrue.eval(xx.reshape(-1,1), log=False), 'g')
    plt.axis([-3, 3, 0., 2.5])
    plt.title(algos[a])
plt.show()        